<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/127_%E7%92%B0%E5%A2%83%E3%81%A8%E3%82%B3%E3%83%B3%E3%83%95%E3%82%A3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

環境とコンフィグ
================

設定の優先順位
--------------

環境やユーザーの設定に応じてプログラムの動作が切り替わるようにすれば、プログラムはより広く使われるようになる。

プログラムがユーザーから値を受ける方法は何通りもあり、いくつかの方法に対応して優先順位を付けることが多く行われる。優先順位の付け方はプログラマーが自由に決めてよいが、一時的に使用される場所やアクセスしやすい場所の設定を優先するほうが直感的である。実際、多くの有名なプログラムでも、この方針で以下のような優先順位が採用されている:

  1. コマンドライン引数
  2. コマンドライン引数で明示的に指定された設定ファイル
  3. 環境変数
  4. カレントディレクトリにある設定ファイル
  5. ホームディレクトリまたは OS が推奨するアプリケーション用ディレクトリにある設定ファイル

環境に関する情報
----------------

### 実行中のプラットフォーム ###

`sys.platform` は、実行中のプラットフォーム（OS）を識別する文字列である（`sys.path` にプラットフォーム固有のサブディレクトリを追加することに利用されている）。

| システム | `platform` の値 |
|:---|:---|
| AIX | `'aix'` |
| Emscripten | `'emscripten'` |
| Linux | `'linux'` |
| WASI | `'wasi'` |
| Windows | `'win32'` |
| Windows/Cygwin | `'cygwin'` |
| macOS | `'darwin'` |

In [ ]:
import sys
sys.platform

'linux'

ちなみに、`os.name` は、Python 実装のコンパイル時に組み込まれたモジュールのうち、OS に依存するモジュールの名前（`'posix'`, `'nt'`, `'java'` のいずれか）である。

In [ ]:
import os
os.name

'posix'

OS の情報が場合分けに必要なわけではなく、単に OS の名前やバージョン表記を取得したいのであれば、`platform.system()`, `platform.release()`, `platform.version()` が使える。これらは不明な場合に空文字列を返すことに注意。

In [ ]:
import platform
platform.system(), platform.release(), platform.version()

('Linux', '6.1.85+', '#1 SMP PREEMPT_DYNAMIC Sun Apr 28 14:29:16 UTC 2024')

### コンピュータのネットワーク名 ###

`platform.node()` は、コンピュータのネットワーク名を返す。これも不明な場合は空文字列を返す。

In [ ]:
import platform
platform.node()

'bcc35b63d48d'

### 実行中の Python インタープリター ###

`sys.version_info` は、Python のバージョン番号を示す 5 要素の名前付きタプル: `(major, minor, micro, releaselevel, serial)`。`releaselevel` 以外は全て整数である。

In [ ]:
import sys
sys.version_info

sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)

Python のバージョンを文字列で取得したいのであれば、`platform.python_version()` が使える。

In [ ]:
import platform
platform.python_version()

'3.10.12'

`sys.hexversion` は、単精度整数にエンコードされたバージョン番号。この値は `hex()` 関数を使って 16 進数に変換することで意味が明らかになる。たとえば、バージョン 3 以上であれば `0x03000000` 以上の値となる。バージョン 3.10 以上であれば `0x030a0000` 以上の値となる。バージョン 3.10.3 以上であれば `0x030a0300` 以上の値となる。

In [ ]:
import sys
hex(sys.hexversion)

'0x30a0cf0'

`sys.implementation` は、モジュール定数で、実行中の Python インタープリターに関する情報が格納されたオブジェクトとなる。以下の属性を持つ。

| 属性 | 意味 |
|:---|:---|
| `name` | 実装の識別子 |
| `version ` | `sys.version_info` と同じ |
| `hexversion` | `sys.hexversion` と同じ |
| `cache_tag` | キャッシュされたモジュールのファイル名に使用されるタグ |

In [ ]:
import sys
sys.implementation

namespace(name='cpython',
          cache_tag='cpython-310',
          version=sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0),
          hexversion=50990320,
          _multiarch='x86_64-linux-gnu')

`sys.flags` は、コマンドラインフラグの状態を表す名前付きタプル。各属性は読み出し専用になっている。属性名とフラグの対応は[公式ドキュメント](https://docs.python.org/ja/3/library/sys.html#sys.flags)を参照。

In [ ]:
import sys
sys.flags

sys.flags(debug=0, inspect=0, interactive=0, optimize=0, dont_write_bytecode=0, no_user_site=0, no_site=0, ignore_environment=0, verbose=0, bytes_warning=0, quiet=0, hash_randomization=1, isolated=0, dev_mode=False, utf8_mode=0, warn_default_encoding=0, int_max_str_digits=-1)

コマンドライン引数
------------------

### コマンドライン構文 ###

以降は、Unix 系 OS でのコマンドライン構文である。Windows でも使える。

``` text
python script.py arg1 arg2 arg3
```

スペースで区切って並べた `arg1`、`arg2`、`arg3` は**位置引数**と呼ばれる。位置引数は並べる順番により識別される。位置引数のリストは可変長とすることもできる。

``` text
python script.py -o1 value1 -o2 --option3 value3
```

`-` を接頭辞として名前を書いた引数、および、`--` を接頭辞として名前を書いた引数は、**オプション引数**と呼ばれる。オプション引数は名前により識別される。スペースを置いて指定した値を格納するもの（`-o1` および `--option3`）と、フラグとするもの（`-o2`）がある。オプション引数は省略可能でなければならない。`-` を付けたほうを短縮名、`--` を付けたほうを正式名として 1 つのオプション引数と扱うことができる。たとえば、`-o1` を短縮名とし、`--option1` を正式名とするとき、コマンドライン上では `-o1` を指定しても `--option1` を指定しても同等とされる。

スクリプトの使い方を説明するヘルプメッセージを表示するオプション引数は、短縮名を `-h`、正式名を `--help` とする。

``` text
python script.py --option4 val1 val2 val3
```

オプション引数の後ろにスペースで区切って並べた `val1`、`val2`、`val3` は、リストとしてオプション引数 `--option4` の値とされる。

### argv ###

`sys.argv` は、Python スクリプトに渡されたコマンドライン引数を単純にスペースで分割した文字列のリストである。その最初の要素 `sys.argv[0]` は、Python インタープリターに渡すインターフェースオプションにより異なる。2 つの関係は、次のとおり。

| オプション | `sys.argv[0]` |
|:---|:---|
| `-c <command>` | `'-c'` |
| `-m <module-name>` | モジュールファイルのフルパス |
| `<python-file>` | `<python-file>` |
| `<directory>` | `<directory>` |
| `<zip-file>` | `<zip-file>` |
| `-` | `'-'` |

In [ ]:
import sys
sys.argv

['/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py',
 '-f',
 '/root/.local/share/jupyter/runtime/kernel-48dd642f-a43a-46ee-bf2b-4e61c5d0deac.json']

### argparse ###

引数を単純にリストで取得するのではなく、コマンドライン構文として解析するには、標準ライブラリの `argparse` モジュールを使用する。

`argparse` の基本的な使い方は、コマンドライン構文を解析するパーサーを `argparse.ArgumentParser` クラスのインストタンスとして作成し、その `add_argument()` メソッドでパーサーにコマンドライン引数の情報を与えることである。次は、もっとも簡単な例である。

``` python
import argparse
parser = argparse.ArgumentParser()
parser.parse_args()
```

下記がこのコードを実行した結果である:

``` text
$ python prog.py
$ python prog.py --help
usage: prog.py [-h]

options:
  -h, --help  show this help message and exit
$ python prog.py --foo
usage: foo.py [-h]
foo.py: error: unrecognized arguments: --foo
$ python prog.py foo
usage: prog.py [-h]
prog.py: error: unrecognized arguments: foo
```

  * コマンドライン引数なしの場合、何も起こらない。
  * コマンドライン引数 `--help`（または `-h`）がある場合、`add_argument()` で `--help` の情報が与えられていないにもかかわらず、スクリプトの使い方（ヘルプメッセージ）が標準出力に表示される。
  * `add_argument()` で情報が与えられていなかったコマンドライン引数が使用されている場合、引数が不正であることを示すエラーメッセージが標準エラー出力に表示される。

ヘルプメッセージもエラーメッセージも `argparse` が自動的に生成したものである。

また、`argparse.ArgumentParser` は `--help` が指定されると、ヘルプメッセージを表示した後 `sys.exit(0)` を呼び出す。そのため、`--help` を使うとプログラムは正常終了する。

例えば、`prog.py` ファイルが以下のようである場合:

``` python
import argparse
parser = argparse.ArgumentParser()
parser.parse_args()
print("スクリプトが続行されました")
```

このスクリプトを `python script.py --help` で実行すると、以下のような出力が表示され、その後スクリプトは終了する:

``` text
$ python test.py --help
usage: test.py [-h]

options:
  -h, --help  show this help message and exit
```

`'スクリプトが続行されました'` は表示されず、スクリプトは `sys.exit(0)` によって終了する。

``` python
argparse.ArgumentParser(prog=None, usage=None, description=None, epilog=None, parents=[], formatter_class=argparse.HelpFormatter, prefix_chars='-',
                        fromfile_prefix_chars=None, argument_default=None, conflict_handler='error', add_help=True, allow_abbrev=True, exit_on_error=True)
```

`argparse.ArgumentParser` クラスのコンストラクタ引数は、主にヘルプメッセージを生成するために使用される。

| 引数 | 意味 |
|:---|:---|
| `prog` | プログラム名を指定する。`None` の場合は `os.path.basename(sys.argv[0])` が使われる |
| `usage` | プログラムの利用方法を文字列で指定する。`None` の場合はパーサーに追加された引数から生成される |
| `description` | 引数のヘルプの前に表示されるテキストを指定する。`None` の場合はテキストなし |
| `epilog` | 引数のヘルプの後に表示されるテキストを指定する。`None` の場合はテキストなし |
| `parents` | `ArgumentParser` オブジェクトのリストを指定する。このリストに含まれるオブジェクトの引数が追加される |
| `formatter_class` | ヘルプメッセージの整形をカスタマイズするためのクラスを指定する。デフォルトは `argparse.HelpFormatter` |
| `prefix_chars` | オプション引数の接頭辞になる文字集合を指定する。デフォルトは `'-'`。たとえば `'+'` を指定すると `+o` のような引数になる |
| `fromfile_prefix_chars` | ファイルから追加の引数を読み込む場合に、そのファイルの名前の接頭辞になる文字列を指定する。`None`（デフォルト）の場合は接頭辞なし。たとえば `'@'` を指定<br />すると `@file.txt` のようにファイルを指定できる |
| `argument_default` | 全体的に適用される引数のデフォルト値を指定する。デフォルトは `None` |
| `conflict_handler` | すでに利用されているオプション文字列を使って新しい引数を作ろうとしたときの挙動を以下の文字列で指定する<br /><br />・`'error'`: `argparse.ArgumentError` 例外を送出する（デフォルト）<br /><br />・`'resolve'`: オプション文字列が上書きされる |
| `add_help` | `True`（デフォルト）の場合、`-h` 引数および `--help` 引数をパーサーに追加する |
| `allow_abbrev` | `True`（デフォルト）の場合、正式名の引数を先頭の 1 文字の短縮名で指定できるようにする |
| `exit_on_error` | `False` の場合、`--help` 実行時に `sys.exit()` せずに `argparse.ArgumentError` 例外を送出する（プログラム内で捕捉できる） |

以下は、`argparse.ArgumentParser` のメソッド。

``` python
ArgumentParser.parse_args(args=None, namespace=None)
```

このメソッドは、コマンドライン引数を解析し、`argparse.Namespace` オブジェクトを作成して返す。パーサーは、コマンドライン引数の解析結果を `argparse.Namespace` オブジェクトに属性を追加する形で保存する。

`parse_args()` メソッドの引数は、次のとおり。

| 引数 | 意味 |
|:---|:---|
| `args` | 解析する文字列のリストを指定する。`None`（デフォルト）なら、`sys.argv` から取得される |
| `namespace` | 戻り値のために使用する `argparse.Namespace` オブジェクトを指定する。`None`（デフォルト）なら、新しい空の `argparse.Namespace` オブジェクトが作成される |

``` python
ArgumentParser.add_argument(name or flags...[, action][, nargs][, const][, default][, type][, choices][, required][, help][, metavar][, dest])
```

このメソッドは、パーサーにコマンドライン引数の情報を与える。パーサーは、与えられた情報に基づいてコマンドライン引数を解析する。

`name or flags...` 以外はすべてキーワード引数である。

| 引数 | 意味 |
|:---|:---|
| `name or flags...` | `'foo'` のように位置引数の名前か、`'-f', '--foo'` のようにオプション引数の名前のリストを指定する |
| `dest` | `Namespace` に追加される属性の名前を指定する。省略する場合は、`name or flags...` が使われる |
| `action` | 引数をどのように処理するかを指定する<br /><br />・`'store'`: 引数に与えられた値を `Namespace` の属性に格納する（デフォルト）<br /><br />・`'store_const'`: 引数はフラグとなり、引数が指定されていれば `const` キーワード引数で指定された値を `Namespace` の属性に格納する<br /><br />・`'store_true'`: 引数はフラグとなり、引数が指定されていれば `True` を `Namespace` の属性に格納する。デフォルト値は `False` とする<br /><br />・`'store_false'`: 引数はフラグとなり、引数が指定されていれば `False` を `Namespace` の属性に格納する。デフォルト値は `True` とする<br /><br />・`'append'`: 引数は複数回使用可能で、それぞれ与えられた値を `Namespace` の属性値であるリストに追加する。たとえばコマンドライン引数が `--foo 1 --foo 2` なら属性<br />　 `foo` が追加されて値が `['1', '2']` となる<br /><br />・`'append_const'`: 引数は複数回使用可能で、`const` キーワード引数で指定された値を `Namespace` の属性値であるリストに追加する<br /><br />・`'extend'`: 引数は複数回使用可能で、それぞれ与えられた値で `Namespace` の属性値であるリストを拡張する。たとえばコマンドライン引数が `--foo f1 --foo f2 f3 f4` <br />　なら属性 `foo` が追加されて値が `['f1', 'f2', 'f3', 'f4']` となる<br /><br />・`'count'`: 引数の数を `Namespace` の属性に格納する。たとえば、名前を `'--verbose'` と `'-v'` とする引数についてコマンドライン引数が `-vvv` なら属性 `verbose` が追加<br />　されて値が `3` となる<br /><br />・`'help'`: 引数はヘルプメッセージを表示し、終了する<br /><br />・`'version'`: 引数は `version` キーワード引数に指定したバージョン情報を表示して終了する<br /><br />`'help'`、`'version'` の場合には、`Namespace` に属性が追加されない |
| `nargs` | 指定する値によってコマンドライン引数が以下のように解釈される:<br /><br />・`N`（整数）: 引数の後ろの `N` 個の要素がコマンドラインから集められ、リストに格納される。たとえば `--foo` について `nargs=2` としコマンドライン引数が `--foo a b c` な<br />　ら属性 `foo` が追加されて値が `['a', 'b']` となる（`c` は別の位置引数と解釈される）<br /><br />・`'?'`: 引数の後ろの 1 個の要素が値に使われるが、すぐ後ろが別のオプション引数ならデフォルト値が使われる<br /><br />・`'*'`: 引数の後ろのすべての要素がコマンドラインから集められ、リストに格納される<br /><br />・`'+'`: `'*'` と同様であるが、加えて、最低でも 1 つのコマンドライン引数が存在しない場合にエラーメッセージを生成する |
| `default` | コマンドラインで値が渡されなかった場合のデフォルト値を指定する |
| `type` | 引数に渡された値を指定した型に変換する。デフォルトでは `str`。ユーザーが定義した関数（や呼び出し可能オブジェクト）も使用できる |
| `choices` | 引数の値として許される値を格納したコンテナ型（`list` など）を指定する |
| `required` | `True` を指定すると、引数は省略できない（オプション引数のみ有効） |
| `help` | 引数の簡潔な説明を指定する |
| `metavar` | ヘルプメッセージの中で引数を参照するときに使われる名前を指定できる。省略する場合、デフォルトでは、位置引数には `dest` の値をそのまま利用し、オプション引数<br />　については `dest` の値を大文字に変換して利用する |

``` python
ArgumentParser.print_help(file=None)
```

このメソッドは、ヘルプメッセージを表示する。`file` が `None` の場合、 `sys.stdout` に出力される。

### 位置引数の利用 ###

以降では、Colab 上で動作を確認するため、`parser.parse_args()` に引数のリストを渡している。ふつうにコマンドライン引数を解析するには、`parser.parse_args()` を引数なしで呼び出すこと。

以下は位置引数の簡単な例である。

In [ ]:
import argparse

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("echo")

    # python prog.py foo と同じ
    args = parser.parse_args(["foo"])
    print(f"echo 引数の値は {args.echo!r}")
    print("-" * 50)

    # python prog.py --help と同じ
    parser.print_help()

if __name__ == "__main__":
    main()

echo 引数の値は 'foo'
--------------------------------------------------
usage: colab_kernel_launcher.py [-h] echo

positional arguments:
  echo

options:
  -h, --help  show this help message and exit


コマンドラインで与えた位置引数 `foo` は、`parser.parse_args()` が返す `Namespace` オブジェクトの `echo` 属性に格納されることがわかる。ちなみに、コマンドラインで位置引数を与えないと、`error: the following arguments are required: echo` のようなエラーメッセージが表示されて終了する。

このヘルプメッセージは、`echo` が位置引数であることだけしかわからないものなので、`echo` の説明を加える:

In [ ]:
import argparse

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("echo", help="echo the string you use here")

    # python prog.py foo と同じ
    args = parser.parse_args(["foo"])
    print(f"echo 引数の値は {args.echo!r}")
    print("-" * 50)

    # python prog.py --help と同じ
    parser.print_help()

if __name__ == "__main__":
    main()

echo 引数の値は 'foo'
--------------------------------------------------
usage: colab_kernel_launcher.py [-h] echo

positional arguments:
  echo        echo the string you use here

options:
  -h, --help  show this help message and exit


デフォルトでは引数は文字列として格納されるので、他の型が必要な場合はパーサーに伝える必要がある:

In [ ]:
import argparse

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("square", help="display a square of a given number", type=int)

    # python prog.py 4 と同じ
    args = parser.parse_args(["4"])
    print(args.square**2)

if __name__ == "__main__":
    main()

16


このコードで` parser.parse_args(["four"])` （コマンドラインなら `python prog.py four`）のように `int` 型に変換できない引数を与えると、`error: argument square: invalid int value: 'four'` のようなエラーメッセージが表示されて終了する。

### オプション引数の利用 ###

以下はオプション引数の例である。

In [ ]:
import argparse

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("-o1", "--option1", required=True)
    parser.add_argument("-o2", action="store_true")
    parser.add_argument("--option3", type=int, choices=[1, 2, 3], default="adult")
    parser.add_argument("-o4", "--option4", nargs=2)

    # python prog.py -o1 foo --option3 3 -o4 spam eggs と同じ
    args = parser.parse_args(["-o1", "foo", "--option3", "3", "-o4", "spam", "eggs"])
    assert args.option1 == "foo"
    assert args.o2 is False  # デフォルト値
    assert args.option3 == 3
    assert args.option4 == ["spam", "eggs"]

if __name__ == "__main__":
    main()

`-o1` や `-o4` のように、短縮名と正式名の両方を指定することができ、その場合には正式名のほうで属性が追加されることに注意する。

`-o1` については、`required=True` を指定しているので省略できない。もしコマンドラインで省略されていた場合、`error: the following arguments are required: -o1/--option1` のようなエラーメッセージが表示されて終了する。

`--option3` については、`choices` 引数を指定しているので、リスト以外の要素（たとえば `4`）をコマンドラインで与えると、`error: argument --option3: invalid choice: 4 (choose from 1, 2, 3)` のようなエラーメッセージが表示されて終了する。

以下は、バージョンを表示するオプション引数の例である。

``` python
import argparse
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--version', action='version', version='%(prog)s 2.0')
```

下記がこのコードを実行した結果である:

``` text
$ python prog.py --version
PROG 2.0
```

### FileType オブジェクト ###

`argparse` では、 `ArgumentParser.add_argument()` の `type` 引数に渡すことができる `argparse.FileType` オブジェクトが利用できる。

``` python
argparse.FileType(mode='r', bufsize=-1, encoding=None, errors=None)
```

`argparse.FileType` オブジェクトは呼び出し可能であり、パスを渡して呼び出すとファイルオブジェクトを返す。コンストラクタ引数の意味は、`open()` 関数のものと同じ。

以下は、`argparse.FileType` の使用例である。

In [ ]:
import argparse
import sys

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("infile", nargs="?", type=argparse.FileType("r", encoding="UTF-8"), default=sys.stdin)
    parser.add_argument("outfile", nargs="?", type=argparse.FileType("w", encoding="UTF-8"), default=sys.stdout)

    # python prog.py sample_data/README.md と同じ
    args = parser.parse_args(["sample_data/README.md"])
    print(args.infile)
    print(args.outfile)

if __name__ == "__main__":
    main()

<_io.TextIOWrapper name='sample_data/README.md' mode='r' encoding='UTF-8'>


コマンドラインで与えた第 1 引数 `sample_data/README.md` は、そのパスから生成されるファイルオブジェクトに変換されて格納されることがわかる。これは、位置引数を文字列として格納し `open()` 関数に渡して開いてもほぼ同じことなのだが、存在しないパスを指定すると `FileNotFoundError` 例外が発生するのではなく、エラーメッセージが表示されて終了する点が異なる。つまり、`argparse.FileType` を使用する場合、`open()` 関数の呼び出しと例外処理のコードを書く必要がなくなる。

2 つの位置引数は、それぞれ `add_argument()` に `nargs='?` と `default=sys.stdin` を渡している。これにより、位置引数が省略可能で、デフォルトでは標準入力と標準出力が格納される。実際、コマンドラインで第 2 引数 `outfile` を与えていないので、標準出力（Colab 上では `ipykernel.iostream.OutStream`）が格納されている。これらの引数を指定していない場合は、位置引数を省略するとエラーメッセージが表示されて終了する。

`FileType` オブジェクトは擬似引数 `'-'` を識別し、読み込み用の `FileType` であれば `sys.stdin` を、書き込み用の `FileType` であれば `sys.stdout` に変換する:

In [ ]:
import argparse

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("infile", type=argparse.FileType(encoding="UTF-8"))

    # python prog.py - と同じ
    args = parser.parse_args(["-"])
    print(args.infile)

if __name__ == "__main__":
    main()

<_io.TextIOWrapper name='<stdin>' mode='r' encoding='utf-8'>


### サブコマンド ###

多くのプログラムは、その機能をサブコマンドへと分割する。たとえば、パッケージ管理システム `pip` は、`pip list`、`pip freeze`、`pip install` などのサブコマンドを利用でき、サブコマンドごとに異なるコマンドライン引数を必要とする。

`ArgumentParser` インスタンスは `add_subparsers()` メソッドによりサブコマンドをサポートしている。`add_subparsers()` メソッドは、特殊なアクションオブジェクトを返す。このオブジェクトには 1 つのメソッド `add_parser()` を持ち、コマンド名と `ArgumentParser` コンストラクタの任意の引数を受け取り、通常の方法で操作できる `ArgumentParser` オブジェクトを返す。

In [ ]:
import argparse

def main():
    # トップレベルのパーサーの作成
    parser = argparse.ArgumentParser(prog="PROG")
    subparsers = parser.add_subparsers()

    # "a" コマンドのためのパーサーを作成
    parser_a = subparsers.add_parser("a", help="a help")
    parser_a.add_argument("bar", type=int, help="bar help")

    # "b" コマンドのためのパーサーを作成
    parser_b = subparsers.add_parser("b", help="b help")
    parser_b.add_argument("--baz", choices="XYZ", help="baz help")

    # python prog.py a 12 と同じ
    args_a = parser.parse_args(["a", "12"])
    assert args_a.bar == 12

    # python prog.py b --baz Z と同じ
    args_b = parser.parse_args(["b", "--baz", "Z"])
    assert args_b.baz == "Z"

if __name__ == "__main__":
    main()

`python prog.py a --help` のようにサブコマンドごとにヘルプメッセージを出力させることもサポートされる。

`add_subparsers()` の引数:

| 引数 | 意味 |
|:---|:---|
| `title` | ヘルプ出力でのサブパーサーグループのタイトル。デフォルトは、`description` が指定されている場合は `'subcommands'` に、指定されていない場合は位置引数のタイトルになる |
| `description` | ヘルプ出力に表示されるサブパーサーグループの説明。デフォルトは `None` |
| `prog` | サブコマンドのヘルプに表示される使用方法の説明。デフォルトではプログラム名と位置引数の後ろに、サブパーサーの引数が続く |
| `parser_class` | サブパーサーのインスタンスを作成するときに使用されるクラス。デフォルトでは現在のパーサーのクラス（例: `ArgumentParser`）になる |
| `action` | コマンドラインにこの引数があったときの基本のアクション |
| `dest` | サブコマンド名を格納する属性の名前。デフォルトは `None` で値は格納されない |
| `required` | サブコマンドが必須であるかどうかを指定する。デフォルトは `False` |
| `help` | ヘルプ出力に表示されるサブパーサーグループのヘルプ。デフォルトは `None` |
| `metavar` | 利用可能なサブコマンドをヘルプ内で表示するための文字列。デフォルトは `None` で、サブコマンドを `{cmd1, cmd2, ..}` のような形式で表す |

環境変数
--------

``` python
os.environ
```

`os.environ` は、環境変数を表現するマッピングオブジェクトである。`os.environ` には、最初に `os` モジュールがインポートされたタイミングの環境変数が格納される。「最初に」というのは、通常は Python の起動時に `site.py` が処理されるタイミングである。それ以後に外部から変更された環境変数は `os.environ` には反映されない。逆に、`os.environ` の変更は、実行中のプロセスにのみ反映され、別のプロセスやシステムとの間で値を共有できない。

``` python
os.getenv(key, default=None)
```

この関数は、環境変数名 `key` の値を返す。戻り値は常に文字列である。名前 `key` を持つ環境変数が存在しないときは、この関数は `default` 引数に設定した値（`default` を設定しない場合は `None`）を返す。`getenv()` は `os.environ` を使用するため、インポート後に外部から行われた環境変数の変更を反映しないことに注意。

In [ ]:
import os
os.environ['monty'] = 'python'
assert os.environ.get('monty') == 'python'
os.getenv('PYTHONPATH'), os.getenv('LANG')

('/env/python', 'en_US.UTF-8')

設定ファイル
------------

### configparser ###

標準ライブラリの `configparser` モジュールは、INI ファイルを読み込む機能を提供する。

次は、INI ファイルの例である。

``` ini
[DEFAULT]
home_dir = /Users/User
limit = 100
debug = No

[Multiline Values]
chorus = I'm a lumberjack, and I'm okay
    I sleep all night and I work all day

# comment
; comment

[WORK]
data_dir = %(home_dir)s/AppData
```

`[]` で囲んだ行がセクションの始まりを表し、`[]` の中身がセクション名となる。セクションに `key = value` を書いて、キーと値を指定する。`=` の代わりに `:` を使ってもよい。値として複数行の文字列を指定する場合は、2行目以降を改行してインデントを付ければよい。

行の先頭に `#` または `;` を置くと、その行はコメントとして扱われる。

一度定義したキーを `%()s` で囲んで埋め込むと、キーの値を利用できる。

INI ファイルを読み込む手順は、以下のようになる。

  1. `configparser.ConfigParser()` コンストラクタの呼び出しで、パーサーを作成する。
  2. パーサーの `read(filename, encoding=None)` メソッドで、INI ファイルを読み込む。`filename` は path-like オブジェクト。
  3. パーサーに対して、辞書のように `[SECTION]` でセクションを参照するオブジェクトを得る。
  4. セクションオブジェクトに対して、辞書のように `[key]` で値を文字列として取得する。

``` python
import configparser
config = configparser.ConfigParser()
config.read("./config.ini", encoding="utf-8")
default = config["DEFAULT"]
home_dir = default["home_dir"]
# home_dir = config["DEFAULT"]["home_dir"] のような書き方もできる
```

値を数値として取得したい場合は、組み込み関数 `int()` または `float()` で型変換する必要がある。

値を `bool` 型で取得する場合は、パーサーのメソッド `getboolean(section)` が使える。このメソッドは、値の `'yes'` および `'true'`、`'on'`、`'1'` を `True` に変換し、`'no'` および `'false'`、`'off'`、`'0'` を `False` に変換する。

``` python
debug = config["DEFAULT"].getboolean("debug")
```

### tomllib ###

標準ライブラリの `tomllib` モジュールは、TOML を解析する機能を提供する（Python 3.11 で追加）。このモジュールは、TOML の書き出しをサポートしていない。

``` python
tomllib.load(fp, /, *, parse_float=float)
```

この関数は、TOMLファイルを読み込み、解析結果の辞書を返す。第 1 引数には読み込み可能なバイナリモードのファイルオブジェクトを指定する。TOML の型は以下の変換表を使用して Python のデータ型に変換される。

| TOML | Python |
|:---|:---|
| TOMLドキュメント | 辞書 |
| 文字列 | `str` |
| 整数 | `int` |
| 浮動小数点数 | `float` （`parse_float` で設定可能） |
| ブール値 | `bool` |
| オフセット付き日時 | `datetime.datetime`（`tzinfo` 属性は `datetime.timezone` のインスタンスが設定される） |
| ローカルの日時 | `datetime.datetime` （`tzinfo` 属性は `None` に設定される） |
| ローカルの日付 | `datetime.date` |
| ローカルの時刻 | `datetime.time` |
| 配列 | `list` |
| テーブル | `dict` |
| インラインテーブル | `dict` |
| テーブルの配列 | `list[dict]` |

TOML の浮動小数点数型に対しては、`parse_float` に指定された呼び出し可能オブジェクトが呼び出されて変換が行われる。たとえば `decimal.Decimal` を指定すれば 10 進数型への変換となる。

無効な TOML ドキュメントの場合は `tomllib.TOMLDecodeError` 例外が送出される。

TOML ファイルからデータを取得するコードは、次のようになる。

``` python
import tomllib
with open("pyproject.toml", "rb") as f:
    data = tomllib.load(f)
```

このように、バイナリモードでファイルオブジェクトを取得する必要がある。

``` python
tomllib.loads(s, /, *, parse_float=float)
```

この関数は、文字列 `s` から TOML を読み込むこと以外は、`load()` と同じである。

``` shell
>>> import tomllib
>>> toml_str = """
... python-version = "3.11.0"
... python-implementation = "CPython"
... """
>>> data = tomllib.loads(toml_str)
>>> data
{'python-version': '3.11.0', 'python-implementation': 'CPython'}
```

サードパーティ製の [Tomli-W](https://github.com/hukkin/tomli-w) パッケージは、 TOML の書き込み用に `tomllib` モジュールと組み合わせて使用でき、標準ライブラリの `pickle` モジュールと同様の書き込み API を提供する。ライセンスは MIT license。インストール方法は次のとおり。

``` shell
pip install tomli-w
```

文字列として保存:  
``` python
import tomli_w

doc = {"table": {"nested": {}, "val3": 3}, "val2": 2, "val1": 1}
expected_toml = """\
val2 = 2
val1 = 1

[table]
val3 = 3

[table.nested]
"""
assert tomli_w.dumps(doc) == expected_toml
```

ファイルへの書き込み:  
``` python
import tomli_w

doc = {"one": 1, "two": 2, "pi": 3}
with open("path_to_file/conf.toml", "wb") as f:
    tomli_w.dump(doc, f)
```

### PyYAML ###

[PyYAML](https://pypi.org/project/PyYAML/) は、YAML 文書を Python のオブジェクト（リストや辞書）に変換するための関数を提供するサードパーティ製パッケージである。ライセンスは MIT license。インストール方法は次のとおり。

``` shell
pip install PyYAML
```

[YAML](https://ja.wikipedia.org/wiki/YAML) は、JSON と同等のデータ構造を人間が読み書きしやすい形式で表現できるデータフォーマットである。最新バージョンは YAML 1.2 であるが、前バージョン YAML 1.1 に対して後方互換性がないことに注意する。PyYAML は YAML 1.1 に対応し、YAML 1.2 には対応していない。

YAML を読み込むには、`yaml.load(stream, Loader)` 関数を使う。第 1 引数には、読み出し可能なファイルオブジェクトか、あるいは文字列を指定する。第 2 引数 `Loader` にローダーを指定する必要がある。

| ローダー | 機能 |
|:---|:---|
| `FullLoader` | YAML を Python の任意のクラスのインスタンスとしてロードできる。YAML を完全にロードできるが、信頼できない YAML 文書に使用する場合には危険である |
| `SafeLoader` | このローダーを使うと、Python オブジェクトの生成機能は `int` や `list` などの単純な型に制限される。信頼できない YAML 文書をロードする場合に推奨される |
| `BaseLoader` | このローダーを使うと、すべてのスカラーは文字列としてロードされる |

`yaml.safe_load(stream)` 関数は、`yaml.load(stream, yaml.SafeLoader)` と同等。

``` python
import yaml
with open('sample.yml', 'r', encoding='utf-8') as f:
     data = yaml.load(f, yaml.FullLoader)
```

`yaml.load()` 関数は、YAML 文書がハッシュ（連想配列）なら `dict` オブジェクトを返し、YAML 文書がリスト（配列）なら `list` オブジェクトを返す。

次のコードは、YAML 文書がブロック形式のハッシュを定義している。`#` から改行まではコメントである。

In [ ]:
import yaml
yaml.safe_load('''
#整数
int1: 2
int2: -100

#浮動小数点数
float1: 0.1

#NULL
nil1: null
nil2: ~

#Boolean (YAML 1.2 では Boolean として扱うのは true, false だけ)
bool1: true
bool2: false
bool3: yes
bool4: no
bool5: on
bool6: off

#日付
birthday: 1990-01-01

#タイムスタンプ
stamp1: 2020-12-01 10:00:00
stamp2: 2020-12-01 10:00:00 +9:00

#文字列
str1: hoge
str2: 'He said "goodbye."'
str3: "spam
ham\
eggs" #二重引用符では改行が可能で、\を付けるとエスケープできる

#リスト-フロー形式
list1: [Java, PHP, Python, [spam, ham, eggs]]

#リスト-ブロック形式
#インデントでネストを表すが、インデントにはタブが使えずスペースのみが使える
list2:
  - Java
  - PHP
  - Python
  - - spam
    - ham
    - eggs

#ハッシュ-フロー形式
hash1: {name: John Smith, age: 33}

#ハッシュ--ブロック形式
#インデントでネストを表すが、インデントにはタブが使えずスペースのみが使える
hash2:
  name: Mary Smith
  age: 27
''')

{'int1': 2,
 'int2': -100,
 'float1': 0.1,
 'nil1': None,
 'nil2': None,
 'bool1': True,
 'bool2': False,
 'bool3': True,
 'bool4': False,
 'bool5': True,
 'bool6': False,
 'birthday': datetime.date(1990, 1, 1),
 'stamp1': datetime.datetime(2020, 12, 1, 10, 0),
 'stamp2': datetime.datetime(2020, 12, 1, 10, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400))),
 'str1': 'hoge',
 'str2': 'He said "goodbye."',
 'str3': 'spam hameggs',
 'list1': ['Java', 'PHP', 'Python', ['spam', 'ham', 'eggs']],
 'list2': ['Java', 'PHP', 'Python', ['spam', 'ham', 'eggs']],
 'hash1': {'name': 'John Smith', 'age': 33},
 'hash2': {'name': 'Mary Smith', 'age': 27}}

次のコードは、YAML 文書がブロック形式のリストを定義している。

In [ ]:
import yaml
yaml.safe_load('''
#リテラル
- 0

#リスト
- [1.5, 2.3, -0.7]

#ハッシュ
- name: Mary Smith
  age: 27
''')

[0, [1.5, 2.3, -0.7], {'name': 'Mary Smith', 'age': 27}]

YAML では、アンカーでデータをマークすると、後でそのデータをエイリアスで参照することができる。アンカーは、 `&アンカー名` の形で、データの前に空白を挟んで記述する。エイリアスは、`*アンカー名` の形で、置いた場所にてデータが参照される。

In [ ]:
import yaml
yaml.safe_load('''
title: マイアプリ
ip: &ip 127.0.0.1
port: 443
debug: true
version: "0.1"
db_ip: *ip
''')

{'title': 'マイアプリ',
 'ip': '127.0.0.1',
 'port': 443,
 'debug': True,
 'version': '0.1',
 'db_ip': '127.0.0.1'}

アンカー / エイリアスは、スカラーだけでなく、リストやハッシュに対して使うこともできる。

In [ ]:
import yaml
yaml.safe_load('''
node1:
  name: one
  options: &options
    request_max_size: 1000
    timeout: 10
    backoff_factor: 0.2

node2:
  name: two
  options: *options
''')

{'node1': {'name': 'one',
  'options': {'request_max_size': 1000, 'timeout': 10, 'backoff_factor': 0.2}},
 'node2': {'name': 'two',
  'options': {'request_max_size': 1000, 'timeout': 10, 'backoff_factor': 0.2}}}

ハッシュを参照するエイリアスを別のハッシュの中に置き、`<<:` で修飾すると、ハッシュをマージする。記述量は減るが、直感的ではなくなることに注意する。

In [ ]:
import yaml
yaml.safe_load('''
node1:
  name: one
  options: &options
    request_max_size: 1000
    timeout: 10

node2:
  name: two
  options:
    <<: *options
    backoff_factor: 0.2
''')

{'node1': {'name': 'one',
  'options': {'request_max_size': 1000, 'timeout': 10}},
 'node2': {'name': 'two',
  'options': {'request_max_size': 1000, 'timeout': 10, 'backoff_factor': 0.2}}}

文字列に `|` でマークすると、インデント以外のすべての文字が文字列の内容と見なされる。改行が `\n` として含まれる。最後の行の行末にも `\n` が付く。

In [ ]:
import yaml
yaml.safe_load('''
title: マイアプリ
description: |
  このアプリは、オンラインで
  YAMLを解析し、エラーも表示
  します
''')

{'title': 'マイアプリ', 'description': 'このアプリは、オンラインで\nYAMLを解析し、エラーも表示\nします\n'}

文字列に `|-` でマークすると、`|` とほとんど同様であるが、最後の行の行末には `\n` が付かない。

In [ ]:
import yaml
yaml.safe_load('''
title: マイアプリ
description: |-
  このアプリは、オンラインで
  YAMLを解析し、エラーも表示
  します
''')

{'title': 'マイアプリ', 'description': 'このアプリは、オンラインで\nYAMLを解析し、エラーも表示\nします'}

文字列に `>` でマークすると、インデント以外の文字が文字列の内容と見なされるが、途中の改行は半角スペースに置き換えられる。最後の行の行末に `\n` が付く。これを使うと、長い行を読みやすいように複数行に記述することができる。

In [ ]:
import yaml
yaml.safe_load('''
title: マイアプリ
description: >
  このアプリは、オンラインで
  YAMLを解析し、エラーも表示
  します
''')

{'title': 'マイアプリ', 'description': 'このアプリは、オンラインで YAMLを解析し、エラーも表示 します\n'}

複数の YAML をハイフン 3 個の行 `---` で区切って 1 文書に結合することができる。`...` で一連のストリームの終わりを示す。複数の YAML を読み込むには、`yaml.load_all(stream, Loader)` 関数を使う。この関数は、各 YAML から変換した Python オブジェクトを 1 つずつ返すようなジェネレーターを返す。`yaml.safe_load_all(stream)` 関数は、`yaml.load_all(stream, yaml.SafeLoader)` と同等。

In [ ]:
import yaml

gen = yaml.safe_load_all('''
order: 1
menu: ham
---
order: 2
menu: egg
''')

for data in gen:
    print(data)

{'order': 1, 'menu': 'ham'}
{'order': 2, 'menu': 'egg'}


複数の YAML が埋め込まれた文書を `yaml.load()` 関数や `yaml.safe_load()` 関数でロードするとエラーが発生するので、単一の YAML かどうかわからない文書をロードする場合は、`yaml.load_all()` 関数や `yaml.safe_load_all()` 関数を使う。なお、文書の最初の行に `---` を書いて単一の YAML からなる文書であることを示す習慣があり、この場合は `yaml.load()` 関数や `yaml.safe_load()` 関数でロードすることができる。

In [ ]:
import yaml

try:
    data = yaml.safe_load('''
order: 1
menu: ham
---
order: 2
menu: egg
''')
except yaml.composer.ComposerError:
    print("複数 YAML を開くことはできません")

yaml.safe_load('''
---
order: 1
menu: ham
''')

複数 YAML を開くことはできません


{'order': 1, 'menu': 'ham'}

YAML では値は暗黙的に型付けされるが、値の前に `!!型名` という形式のタグを付けることで、明示的な型付けもできる。標準で使用できるタグは[公式ドキュメント](https://yaml.org/type/index.html)を参照。また、 PyYAML は `!!python/tuple` のような独自のタグを定義して Python 固有の型を明示できるようにしている。ただし、`SafeLoader` を使う場合は、独自のタグを使用するとエラーが発生する。

In [ ]:
import yaml

yaml.load('''
boolean: !!bool "true"
integer: !!int "3"
float: !!float 3
str: !!str 2020-12-01 10:00:00
timestamp: !!timestamp "2020-12-01 10:00:00"
tuple: !!python/tuple [spam, ham, eggs]
''', yaml.FullLoader)

{'boolean': True,
 'integer': 3,
 'float': 3.0,
 'str': '2020-12-01 10:00:00',
 'timestamp': datetime.datetime(2020, 12, 1, 10, 0),
 'tuple': ('spam', 'ham', 'eggs')}

Python のオブジェクトを YAML 文書に変換するには、`yaml.dump(data, stream)` 関数を使う。第 1 引数には、Python のオブジェクトを指定する。第 2 引数には、ファイルオブジェクトを指定する。Python に固有の型のオブジェクトは、独自のタグを付けて出力する。

``` python
with open('output.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data, f)
```

`yaml.safe_dump(data, stream)` 関数は、Python に固有の型のオブジェクトを受け取った場合、独自のタグを付けずに出力する。復元時に Python 固有の型のオブジェクトとして復元されない可能性がある。